In [ ]:
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from mlxtend.plotting import plot_confusion_matrix
import os
import random
import tqdm as tqdm
from shutil import copyfile
from keras.optimizers import SGD
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import keras
from keras.optimizers import Adam
import pandas as pd
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
from matplotlib.image import imread
import pathlib
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split, cross_val_predict, cross_val_score
from sklearn import metrics
import cv2




In [ ]:
def read_dataset(folder):
    directory="/content/drive/MyDrive/uygulamalar/weather_kaggle/dataset/"
    for i in tqdm.tqdm(os.listdir(directory+folder)):
        full_path = os.path.join(directory+folder,i)
        try:
            img = cv2.imread(full_path,1)
            img = cv2.resize(img,(256,256))
        except:
            continue
        images.append(img)
        classes.append(folder)
    return images,classes

images = []
classes= [] 
dataset =[]

images,classes= read_dataset("foggy")
images,classes= read_dataset("sunrise")
images,classes= read_dataset("shine")
images,classes= read_dataset("rainy")
images,classes= read_dataset("cloudy")

In [ ]:
print(classes)

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use("fivethirtyeight")
sns.countplot(classes)
plt.show()

In [ ]:
images =  np.array(images)
classes= np.array(classes)
images.shape,classes.shape


In [ ]:
print(classes)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
classes = le.fit_transform(classes)
lb = LabelBinarizer()
classes_binary = lb.fit_transform(classes)
classes_binary.shape

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
seed = 7
np.random.seed(seed)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)




In [ ]:
confusion_matrix_all=0
sum_loss=0
sum_acc=0

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
vgg = VGG19(weights = "imagenet",include_top=False,input_shape=(256,256,3))

In [ ]:
for layer in vgg.layers:
    layer.trainable = False
    


In [ ]:
for train_index, test_index  in kfold.split(images,classes):
    img_size =256
    images_train, images_test = images[train_index],images[test_index]
    classes_train, classes_test = classes_binary[train_index],classes_binary[test_index]
   

  
    model_vgg =Sequential()
    model_vgg.add(vgg)
    model_vgg.add(Flatten())
    model_vgg.add(Dense(5,activation="softmax"))
    model_vgg.build((None,256,256,3))

    model_vgg.summary()
    model_vgg.compile(optimizer=opt ,loss='categorical_crossentropy', metrics =['accuracy'])


    history =  model_vgg.fit(images_train,classes_train,epochs=15, batch_size=32,verbose=2)
    loss,accuracy = model_vgg.evaluate(images_test,classes_test,verbose=1)
   
    classes_pred = model_vgg.predict_classes(images_test,verbose=1)
  
    confusion_matrix_epoch = confusion_matrix(classes[test_index],classes_pred)
    confusion_matrix_all=confusion_matrix_all+confusion_matrix_epoch
    
    sum_acc=sum_acc+accuracy
    sum_loss=sum_loss+loss


In [ ]:
plot_confusion_matrix(conf_mat = confusion_matrix_all,figsize=(8,7), show_normed = True);

In [ ]:
avrg_acc=sum_acc/5
print(avrg_acc)

In [ ]:
avrg_loss=sum_loss/5
print(avrg_loss)